# Introduction

In this project, you will build a neural network of your own design to evaluate the CIFAR-10 dataset.
Our target accuracy is 70%, but any accuracy over 50% is a great start.
Some of the benchmark results on CIFAR-10 include:

78.9% Accuracy | [Deep Belief Networks; Krizhevsky, 2010](https://www.cs.toronto.edu/~kriz/conv-cifar10-aug2010.pdf)

90.6% Accuracy | [Maxout Networks; Goodfellow et al., 2013](https://arxiv.org/pdf/1302.4389.pdf)

96.0% Accuracy | [Wide Residual Networks; Zagoruyko et al., 2016](https://arxiv.org/pdf/1605.07146.pdf)

99.0% Accuracy | [GPipe; Huang et al., 2018](https://arxiv.org/pdf/1811.06965.pdf)

98.5% Accuracy | [Rethinking Recurrent Neural Networks and other Improvements for ImageClassification; Nguyen et al., 2020](https://arxiv.org/pdf/2007.15161.pdf)

Research with this dataset is ongoing. Notably, many of these networks are quite large and quite expensive to train. 

## Imports

In [2]:
## This cell contains the essential imports you will need – DO NOT CHANGE THE CONTENTS! ##
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## Load the Dataset

Specify your transforms as a list first.
The transforms module is already loaded as `transforms`.

CIFAR-10 is fortunately included in the torchvision module.
Then, you can create your dataset using the `CIFAR10` object from `torchvision.datasets` ([the documentation is available here](https://pytorch.org/docs/stable/torchvision/datasets.html#cifar)).
Make sure to specify `download=True`! 

Once your dataset is created, you'll also need to define a `DataLoader` from the `torch.utils.data` module for both the train and the test set.

In [3]:
# Define transforms
## YOUR CODE HERE ##
transform = transforms.Compose( [transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
   
batch_size = 4

# Create training set and define training dataloader
## YOUR CODE HERE ##
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=2)

# Create test set and define test dataloader
## YOUR CODE HERE ##
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False, num_workers=2)

# The 10 classes in the dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Explore the Dataset
Using matplotlib, numpy, and torch, explore the dimensions of your data.

You can view images using the `show5` function defined below – it takes a data loader as an argument.
Remember that normalized images will look really weird to you! You may want to try changing your transforms to view images.
Typically using no transforms other than `toTensor()` works well for viewing – but not as well for training your network.
If `show5` doesn't work, go back and check your code for creating your data loaders and your training/test sets.

In [4]:
def show5(img_loader):
    dataiter = iter(img_loader)
    
    batch = next(dataiter)
    labels = batch[1][0:5]
    images = batch[0][0:5]
    for i in range(4):
        print(classes[labels[i]])
    
        image = images[i].numpy()
        plt.imshow(np.clip(np.rot90(image.T, k=3), 0, 1)) # modified cause it always gave an error
        plt.show()

In [ ]:
# Explore data
## YOUR CODE HERE ##
show5(testloader) 
#this keeps killing kernel and i searched for a fix but nothing worked

cat


## Build your Neural Network
Using the layers in `torch.nn` (which has been imported as `nn`) and the `torch.nn.functional` module (imported as `F`), construct a neural network based on the parameters of the dataset. 
Feel free to construct a model of any architecture – feedforward, convolutional, or even something more advanced!

In [5]:
## YOUR CODE HERE ##

#using that model in the work space makes it doesn't work idk why
#also when i try to view images in the work space it works and i'm confused
import torch
import torch.nn as nn
import torch.nn.functional as F

class NET(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding=(1,1))
        self.conv2 = nn.Conv2d(6, 16, 5, padding=(1,1))
        self.conv3 = nn.Conv2d(16, 100, 5, padding=(1,1))
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 18)
        self.Dropout = nn.Dropout(0.25)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.Dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.Dropout(x)
        x = x.reshape(-1, 400) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.Dropout(x)
        x = self.fc3(x)
        return x
    
net=NET()

Specify a loss function and an optimizer, and instantiate the model.

If you use a less common loss function, please note why you chose that loss function in a comment.

In [6]:
## YOUR CODE HERE ##
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Running your Neural Network
Use whatever method you like to train your neural network, and ensure you record the average loss at each epoch. 
Don't forget to use `torch.device()` and the `.to()` method for both your model and your data if you are using GPU!

If you want to print your loss during each epoch, you can use the `enumerate` function and print the loss after a set number of batches. 250 batches works well for most people!

In [18]:
step=500

def training():
    loss_history = []

    for epoch in range(4): 
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % step == 0 and i!=0:    
                loss_history.append(running_loss / step)
                print(f"Epoch: {epoch + 1}, Step: {i}, Loss: {running_loss / step}")
                running_loss = 0.0
        print('Finished a loop')
    return loss_history

loss_history=training()



Epoch: 1, Step: 500, Loss: 0.452498865479487
Epoch: 1, Step: 1000, Loss: 0.4659758524665376
Epoch: 1, Step: 1500, Loss: 0.4922930414371804
Epoch: 1, Step: 2000, Loss: 0.49598786061560896
Epoch: 1, Step: 2500, Loss: 0.532858936421806
Epoch: 1, Step: 3000, Loss: 0.5110090735584963
Epoch: 1, Step: 3500, Loss: 0.49050354672479446
Epoch: 1, Step: 4000, Loss: 0.5178628933747241
Epoch: 1, Step: 4500, Loss: 0.570733295944381
Epoch: 1, Step: 5000, Loss: 0.5263640781443101
Epoch: 1, Step: 5500, Loss: 0.5613599447741763
Epoch: 1, Step: 6000, Loss: 0.4864653700090639
Epoch: 1, Step: 6500, Loss: 0.5584794895644009
Epoch: 1, Step: 7000, Loss: 0.5598925064208161
Epoch: 1, Step: 7500, Loss: 0.5593349959936459
Epoch: 1, Step: 8000, Loss: 0.5747011279244761
Epoch: 1, Step: 8500, Loss: 0.5756405516820959
Epoch: 1, Step: 9000, Loss: 0.5775294338309904
Epoch: 1, Step: 9500, Loss: 0.5720036063758307
Epoch: 1, Step: 10000, Loss: 0.6232410051070619
Epoch: 1, Step: 10500, Loss: 0.6019687755033374
Epoch: 1, Ste

Plot the training loss (and validation loss/accuracy, if recorded).

In [ ]:
## YOUR CODE HERE ##
plt.plot(loss_history)
#this keeps crashing the kernel too 

## Testing your model
Using the previously created `DataLoader` for the test set, compute the percentage of correct predictions using the highest probability prediction. 

If your accuracy is over 70%, great work! 
This is a hard task to exceed 70% on.

If your accuracy is under 45%, you'll need to make improvements.
Go back and check your model architecture, loss function, and optimizer to make sure they're appropriate for an image classification task.

In [19]:
IS_GPU = torch.cuda.is_available()

correct = 0
total = 0
net.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        if IS_GPU:
            images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network {100 * correct / total}%')



Accuracy of the network 63.58%


## Saving your model
Using `torch.save`, save your model for future loading.

In [21]:
## YOUR CODE HERE ##
torch.save(net.state_dict(), 'saved_model.pth')

## Make a Recommendation

Based on your evaluation, what is your recommendation on whether to build or buy? Explain your reasoning below.
it depends on the use if a very high accuracy is desired i recommend buying one or building one which will need money too so it depends on the task needed



**Double click this cell to modify it**

